In [1]:
import boto3
from dotenv import load_dotenv
import os
from io import StringIO 
import numpy as np
import joblib
import pandas as pd

from io import BytesIO
# Initialize an S3 client
s3_client = boto3.client('s3')


load_dotenv()

# Access the environment variables for AWS credentials
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION')

s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION
)

# Define the S3 bucket and the file path
bucket_name = 'isbfinanceproject'
def get_file_path(vertical, ticker):
    file_path = f"{vertical}/{ticker}/{ticker}_prediction_data.csv"
    return file_path

# Example usage
vertical = 'Energy'  # Replace with the actual vertical
ticker = 'RELIANCE.NS'       # Replace with the actual ticker

# Get the file path
file_path = get_file_path(vertical, ticker)

# Download the file to the local system
try:
    s3_client.download_file(bucket_name, file_path, f"./{ticker}_prediction_data.csv")
    print(f"File {ticker}_prediction_data.csv downloaded successfully.")
except Exception as e:
    print(f"Error downloading file: {e}")

File RELIANCE.NS_prediction_data.csv downloaded successfully.


In [2]:

features = ['Close', 'Volume', 'ma_7', 'ma_30', 'High', 'Low', 'Open', 'lag_1', 'lag_2', 'lag_7', 'daily_returns', 'volatility']
X_test = pd.read_csv(f"./{ticker}_prediction_data.csv")

X_test = X_test.drop([0, 1])
X_test = X_test.reset_index(drop=True)
X_test = X_test[features].fillna(0)  # Fill missing values if any
X_test = X_test.drop(columns=['Close']) 
X_test.head()

,Volume,ma_7,ma_30,High,Low,Open,lag_1,lag_2,lag_7,daily_returns,volatility
0,0.296731,0.000000,0.0,0.242267,0.257497,0.246341,0.239023,0.219713,0.0,0.562008,0.0
1,0.093573,0.000000,0.0,0.235883,0.251846,0.242466,0.253501,0.239023,0.0,0.448642,0.0
2,0.146457,0.000000,0.0,0.241138,0.255337,0.241348,0.249662,0.253501,0.0,0.492708,0.0
3,0.095338,0.000000,0.0,0.239308,0.260506,0.244837,0.253062,0.249662,0.0,0.506856,0.0
4,0.238515,0.225291,0.0,0.243804,0.262435,0.251373,0.258811,0.253062,0.0,0.468954,0.0


In [3]:
def load_from_s3(file_name, bucket_name):
    s3 = boto3.client('s3')
    buffer = BytesIO()
    s3.download_fileobj(bucket_name, file_name, buffer)
    buffer.seek(0)  # Reset the pointer to the beginning of the file
    model = joblib.load(buffer)
    return model

# Load XGBoost model
xgb_model = load_from_s3('Models/XGB/best_xgb_model.pkl', 'isbfinanceproject')

# Load XGBoost parameters (if needed)
xgb_params = load_from_s3('Models/XGB/xgb_params.pkl', 'isbfinanceproject')

In [4]:
# Define the S3 bucket and the file path
bucket_name = 'isbfinanceproject'
def get_file_path(vertical, ticker):
    file_path = f"{vertical}/{ticker}/{ticker}_data.csv"
    return file_path

# Example usage
vertical = 'Energy'  # Replace with the actual vertical
ticker = 'RELIANCE.NS'       # Replace with the actual ticker

# Get the file path
file_path = get_file_path(vertical, ticker)

# Download the file to the local system
try:
    s3_client.download_file(bucket_name, file_path, f"./{ticker}_data.csv")
    print(f"File {ticker}_data.csv downloaded successfully.")
except Exception as e:
    print(f"Error downloading file: {e}")



df = pd.read_csv(f"./{ticker}_data.csv")
df = df.drop([0, 1])
df = df.reset_index(drop=True)
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
min_close = df['Close'].min()
max_close = df['Close'].max()
df.rename(columns={'Price': 'Date'}, inplace=True)
X_test_recent=df

File RELIANCE.NS_data.csv downloaded successfully.


In [5]:
# Initialize the last known features for recursive forecasting
last_known_features = X_test.iloc[-1].copy()  # Start with the most recent row of features
forecasted_prices_7 = []  # List to store predicted prices
forecast_horizon = 7  # Predict for the next 7 days

for step in range(forecast_horizon):
    # Predict the next price using the model
    next_price = xgb_model.predict(last_known_features.values.reshape(1, -1))[0]
    forecasted_prices_7.append(next_price)
    
    # Update features for the next prediction
    # Update lag features
    last_known_features['lag_7'] = last_known_features['lag_2']
    last_known_features['lag_2'] = last_known_features['lag_1']
    last_known_features['lag_1'] = next_price
    
    # Update moving averages
    recent_prices = [next_price, last_known_features['lag_1'], last_known_features['lag_2'], 
                     last_known_features['lag_7']]
    last_known_features['ma_7'] = sum(recent_prices[-7:]) / min(len(recent_prices), 7)
    last_known_features['ma_30'] = sum(recent_prices[-30:]) / min(len(recent_prices), 30)
    
    # Update daily returns
    last_known_features['daily_returns'] = (next_price - last_known_features['lag_1']) / last_known_features['lag_1']
    
    # Update volatility (e.g., standard deviation of recent prices)
    last_known_features['volatility'] = np.std(recent_prices[-7:])
    
    # Update other features if necessary (Volume, High, Low, etc.)
    # These may need to be predicted or assumed static for simplicity.

# Print the forecasted prices
print("Forecasted Prices for the next 7 days:", forecasted_prices_7)


Forecasted Prices for the next 7 days: [0.7215708, 0.7064247, 0.707549, 0.70634514, 0.70634514, 0.70642567, 0.70642567]


In [6]:
def inverse_min_max_scaling(scaled_value, min_value, max_value):
    return (scaled_value * (max_value - min_value)) + min_value

# Rescale the forecasted prices
forecasted_prices_original_7 = [
    inverse_min_max_scaling(pred, min_close, max_close) for pred in forecasted_prices_7
]


In [7]:
forecasted_prices_original_7

[1267.6920678431215,
 1249.5661202926422,
 1250.911572923982,
 1249.4708928903074,
 1249.4708928903074,
 1249.5672615948424,
 1249.5672615948424]

In [8]:
# Number of days to forecast
forecast_horizon = 30

# Initialize the last known features for recursive forecasting
last_known_features = X_test.iloc[-1].copy()  # Start with the most recent row of features
forecasted_prices_scaled_30 = []  # To store scaled predictions
forecasted_prices_original = []  # To store rescaled predictions

# Ensure min_close and max_close are numeric
min_close = float(min_close)
max_close = float(max_close)

# Recursive Forecast for 30 Days
for step in range(forecast_horizon):
    # Predict the next scaled price
    next_price_scaled = xgb_model.predict(last_known_features.values.reshape(1, -1))[0]
    forecasted_prices_scaled_30.append(next_price_scaled)
    
    # Rescale the predicted price to the original scale
    next_price_original = (next_price_scaled * (max_close - min_close)) + min_close 
    forecasted_prices_original.append(next_price_original)
    
    # Update features for the next prediction
    # Update lag features
    last_known_features['lag_7'] = last_known_features['lag_2']
    last_known_features['lag_2'] = last_known_features['lag_1']
    last_known_features['lag_1'] = next_price_scaled  # Use scaled value for lag
    
    # Update moving averages
    recent_prices = forecasted_prices_scaled_30[-7:]  # Use the last 7 predicted prices
    last_known_features['ma_7'] = sum(recent_prices) / len(recent_prices)
    recent_prices_30 = forecasted_prices_scaled_30[-30:] if len(forecasted_prices_scaled_30) >= 30 else recent_prices
    last_known_features['ma_30'] = sum(recent_prices_30) / len(recent_prices_30)
    
    # Update daily returns
    last_known_features['daily_returns'] = (
        (next_price_scaled - last_known_features['lag_1']) / last_known_features['lag_1']
    ) if last_known_features['lag_1'] != 0 else 0
    
    # Update volatility (e.g., standard deviation of recent prices)
    last_known_features['volatility'] = np.std(recent_prices) if len(recent_prices) > 1 else 0

# Print forecasted prices on both scales
print("Forecasted Prices (Scaled):", forecasted_prices_scaled_30)
print("Forecasted Prices (Original Scale):", forecasted_prices_original)


Forecasted Prices (Scaled): [0.7215708, 0.7064242, 0.707549, 0.70634514, 0.70634514, 0.70642567, 0.70642567, 0.70634514, 0.70634514, 0.70642567, 0.70642567, 0.70634514, 0.70634514, 0.70642567, 0.70642567, 0.70634514, 0.70634514, 0.70642567, 0.70642567, 0.70634514, 0.70634514, 0.70642567, 0.70642567, 0.70634514, 0.70634514, 0.70642567, 0.70642567, 0.70634514, 0.70634514, 0.70642567]
Forecasted Prices (Original Scale): [1267.6920678431215, 1249.5654783101545, 1250.911572923982, 1249.4708928903074, 1249.4708928903074, 1249.5672615948424, 1249.5672615948424, 1249.4708928903074, 1249.4708928903074, 1249.5672615948424, 1249.5672615948424, 1249.4708928903074, 1249.4708928903074, 1249.5672615948424, 1249.5672615948424, 1249.4708928903074, 1249.4708928903074, 1249.5672615948424, 1249.5672615948424, 1249.4708928903074, 1249.4708928903074, 1249.5672615948424, 1249.5672615948424, 1249.4708928903074, 1249.4708928903074, 1249.5672615948424, 1249.5672615948424, 1249.4708928903074, 1249.4708928903074,

In [9]:
X_test

,Volume,ma_7,ma_30,High,Low,Open,lag_1,lag_2,lag_7,daily_returns,volatility
0,0.296731,0.000000,0.000000,0.242267,0.257497,0.246341,0.239023,0.219713,0.000000,0.562008,0.000000
1,0.093573,0.000000,0.000000,0.235883,0.251846,0.242466,0.253501,0.239023,0.000000,0.448642,0.000000
2,0.146457,0.000000,0.000000,0.241138,0.255337,0.241348,0.249662,0.253501,0.000000,0.492708,0.000000
3,0.095338,0.000000,0.000000,0.239308,0.260506,0.244837,0.253062,0.249662,0.000000,0.506856,0.000000
4,0.238515,0.225291,0.000000,0.243804,0.262435,0.251373,0.258811,0.253062,0.000000,0.468954,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1232,0.128451,0.758305,0.829838,0.738483,0.738015,0.741227,0.753289,0.769750,0.785250,0.411761,0.118240
1233,0.052142,0.750933,0.821611,0.725158,0.731265,0.725454,0.734989,0.753289,0.775348,0.441058,0.106180
1234,0.063595,0.742933,0.815043,0.727967,0.731687,0.722586,0.725755,0.734989,0.780863,0.476310,0.090960
1235,0.091702,0.736710,0.808137,0.716175,0.716499,0.718116,0.727051,0.725755,0.750364,0.409433,0.094261


In [10]:
dates=df['Date']

In [11]:
import pandas as pd
from datetime import timedelta

# Assuming `dates` is loaded as a Series of strings
dates = pd.to_datetime(dates)  # Ensure all dates are in datetime format
forecast_horizon_7=7

# Determine the last known date
last_date = dates.iloc[-1]  # Extract the last date

# Forecast horizon (e.g., 30 days)
forecast_horizon = len(forecasted_prices_original)

# Generate forecast dates
forecast_dates = [last_date + timedelta(days=i) for i in range(1, forecast_horizon + 1)]
forecast_dates_7 = [last_date + timedelta(days=i) for i in range(1, forecast_horizon_7 + 1)]
print(f"Last known date: {last_date}")
print(f"Forecast dates: {forecast_dates}")


Last known date: 2024-11-14 00:00:00+00:00
Forecast dates: [Timestamp('2024-11-15 00:00:00+0000', tz='UTC'), Timestamp('2024-11-16 00:00:00+0000', tz='UTC'), Timestamp('2024-11-17 00:00:00+0000', tz='UTC'), Timestamp('2024-11-18 00:00:00+0000', tz='UTC'), Timestamp('2024-11-19 00:00:00+0000', tz='UTC'), Timestamp('2024-11-20 00:00:00+0000', tz='UTC'), Timestamp('2024-11-21 00:00:00+0000', tz='UTC'), Timestamp('2024-11-22 00:00:00+0000', tz='UTC'), Timestamp('2024-11-23 00:00:00+0000', tz='UTC'), Timestamp('2024-11-24 00:00:00+0000', tz='UTC'), Timestamp('2024-11-25 00:00:00+0000', tz='UTC'), Timestamp('2024-11-26 00:00:00+0000', tz='UTC'), Timestamp('2024-11-27 00:00:00+0000', tz='UTC'), Timestamp('2024-11-28 00:00:00+0000', tz='UTC'), Timestamp('2024-11-29 00:00:00+0000', tz='UTC'), Timestamp('2024-11-30 00:00:00+0000', tz='UTC'), Timestamp('2024-12-01 00:00:00+0000', tz='UTC'), Timestamp('2024-12-02 00:00:00+0000', tz='UTC'), Timestamp('2024-12-03 00:00:00+0000', tz='UTC'), Timestamp

In [12]:
dates.dtype
dates.count

<bound method Series.count of 0      2019-11-18 00:00:00+00:00
1      2019-11-19 00:00:00+00:00
2      2019-11-20 00:00:00+00:00
3      2019-11-21 00:00:00+00:00
4      2019-11-22 00:00:00+00:00
                  ...           
1234   2024-11-08 00:00:00+00:00
1235   2024-11-11 00:00:00+00:00
1236   2024-11-12 00:00:00+00:00
1237   2024-11-13 00:00:00+00:00
1238   2024-11-14 00:00:00+00:00
Name: Date, Length: 1239, dtype: datetime64[ns, UTC]>

In [13]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,P/E Ratio,Beta (5Y Monthly),Market Cap,Forward P/E,EPS (TTM),Price.1,Sector,Industry
0,2019-11-18 00:00:00+00:00,646.4679565429688,667.097168,679.3491821289062,665.3599243164062,673.2460327148438,14076051,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
1,2019-11-19 00:00:00+00:00,668.8631591796875,690.206909,692.5613403320312,669.7487182617188,670.6630249023438,30176255,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
2,2019-11-20 00:00:00+00:00,685.6538696289062,707.533508,718.848388671875,705.4991455078125,710.9165649414062,43538120,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
3,2019-11-21 00:00:00+00:00,681.2014770507812,702.938965,711.350830078125,698.8016357421875,706.322021484375,14897371,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
4,2019-11-22 00:00:00+00:00,685.1444702148438,707.007751,717.5225830078125,702.93896484375,704.9962158203125,22352865,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,2024-11-08 00:00:00+00:00,1283.75,1283.750000,1301.6500244140625,1275.0,1297.6500244140625,19814406,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
1235,2024-11-11 00:00:00+00:00,1272.699951171875,1272.699951,1286.0,1267.0,1278.949951171875,9056552,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
1236,2024-11-12 00:00:00+00:00,1274.25,1274.250000,1289.300048828125,1267.5,1275.550048828125,10671091,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing
1237,2024-11-13 00:00:00+00:00,1252.050048828125,1252.050049,1275.449951171875,1249.5,1270.25,14633633,25.250996,0.635,1.715367e+13,19.849545,50.2,1252.05,Energy,Oil & Gas Refining & Marketing


In [14]:
import plotly.graph_objects as go
# Historical Data
historical_data = pd.DataFrame({
    'Date': dates,  # Use fixed datetime dates
    'Price': df['Close']  # Historical prices
})
historical_data = historical_data.iloc[-30:]
# Forecast Data
forecast_data = pd.DataFrame({
    'Date': forecast_dates,  # Generated future dates
    'Price': forecasted_prices_original  # Forecasted prices
})

# Combine data and plot
visualization_df = pd.concat([historical_data, forecast_data])

# Plot code remains the same
fig = go.Figure()
fig.add_trace(go.Scatter(x=historical_data['Date'], y=historical_data['Price'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=forecast_data['Date'], y=forecast_data['Price'], mode='lines', name='Forecasted Prices'))
fig.update_layout(title='30-Day Stock Price Forecast', xaxis_title='Date', yaxis_title='Stock Price')
fig.show()

In [15]:
import plotly.graph_objects as go
# Historical Data
historical_data = pd.DataFrame({
    'Date': dates,  # Use fixed datetime dates
    'Price': df['Close']  # Historical prices
})
historical_data = historical_data.iloc[-7:]
# Forecast Data
forecast_data = pd.DataFrame({
    'Date': forecast_dates_7,  # Generated future dates
    'Price': forecasted_prices_original_7  # Forecasted prices
})

# Combine data and plot
visualization_df = pd.concat([historical_data, forecast_data])

# Plot code remains the same
fig = go.Figure()
fig.add_trace(go.Scatter(x=historical_data['Date'], y=historical_data['Price'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=forecast_data['Date'], y=forecast_data['Price'], mode='lines', name='Forecasted Prices'))
fig.update_layout(title='7-Day Stock Price Forecast', xaxis_title='Date', yaxis_title='Stock Price')
fig.show()

In [16]:
import plotly.graph_objects as go
import pandas as pd

# Historical Data
historical_data = pd.DataFrame({
    'Date': dates,  # Use fixed datetime dates
    'Price': df['Close']  # Historical prices
})
historical_data = historical_data.iloc[-30:]

# Forecast Data
forecast_data = pd.DataFrame({
    'Date': forecast_dates,  # Generated future dates
    'Price': forecasted_prices_original  # Forecasted prices
})

# Combine data and plot
visualization_df = pd.concat([historical_data, forecast_data])

# Plot customization
fig = go.Figure()

# Add historical data trace
fig.add_trace(go.Scatter(
    x=historical_data['Date'],
    y=historical_data['Price'],
    mode='lines',
    name='Historical Prices',
    line=dict(color='cyan', width=2),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Add forecasted data trace
fig.add_trace(go.Scatter(
    x=forecast_data['Date'],
    y=forecast_data['Price'],
    mode='lines',
    name='Forecasted Prices',
    line=dict(color='magenta', width=2, dash='dash'),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Update layout for dark theme and vibrant colors
fig.update_layout(
    title=dict(
        text='30-Day Stock Price Forecast',
        font=dict(size=24, color='white'),
        x=0.5
    ),
    xaxis=dict(
        title='Date',
        showgrid=False,
        gridcolor='gray',
        color='white',
        tickformat='%b %d'
    ),
    yaxis=dict(
        title='Stock Price',
        showgrid=False,
        gridcolor='gray',
        color='white'
    ),
    paper_bgcolor='black',  # Dark background
    plot_bgcolor='black',
    legend=dict(
        font=dict(size=12, color='white'),
        bgcolor='rgba(0,0,0,0.5)'
    )
)

# Show the plot
fig.show()


In [17]:
import plotly.graph_objects as go
import pandas as pd

# Historical Data
historical_data = pd.DataFrame({
    'Date': dates,  # Use fixed datetime dates
    'Price': df['Close']  # Historical prices
})
historical_data = historical_data.iloc[-7:]

# Forecast Data
forecast_data = pd.DataFrame({
    'Date': forecast_dates_7,  # Generated future dates
    'Price': forecasted_prices_original_7  # Forecasted prices
})

# Combine data and plot
visualization_df = pd.concat([historical_data, forecast_data])

# Plot customization
fig = go.Figure()

# Add historical data trace
fig.add_trace(go.Scatter(
    x=historical_data['Date'],
    y=historical_data['Price'],
    mode='lines',
    name='Historical Prices',
    line=dict(color='cyan', width=2),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Add forecasted data trace
fig.add_trace(go.Scatter(
    x=forecast_data['Date'],
    y=forecast_data['Price'],
    mode='lines',
    name='Forecasted Prices',
    line=dict(color='magenta', width=2, dash='dash'),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Update layout for dark theme and vibrant colors
fig.update_layout(
    title=dict(
        text='7-Day Stock Price Forecast',
        font=dict(size=24, color='white'),
        x=0.5
    ),
    xaxis=dict(
        title='Date',
        showgrid=False,
        gridcolor='gray',
        color='white',
        tickformat='%b %d'
    ),
    yaxis=dict(
        title='Stock Price',
        showgrid=False,
        gridcolor='gray',
        color='white'
    ),
    paper_bgcolor='black',  # Dark background
    plot_bgcolor='black',
    legend=dict(
        font=dict(size=12, color='white'),
        bgcolor='rgba(0,0,0,0.5)'
    )
)

# Show the plot
fig.show()


In [18]:
import plotly.graph_objects as go
import pandas as pd

# Historical Data
historical_data = pd.DataFrame({
    'Date': dates,  # Use fixed datetime dates
    'Price': df['Close']  # Historical prices
})
historical_data = historical_data.iloc[-30:]

# Forecast Data
forecast_data = pd.DataFrame({
    'Date': forecast_dates,  # Generated future dates
    'Price': forecasted_prices_original  # Forecasted prices
})

# Example proportions for Buy, Sell, Hold
historical_prices = historical_data['Price'].values
predicted_prices = forecast_data['Price'].values
threshold = 0.05  # 2% threshold

# Compute buy, sell, hold
buy_signals = (predicted_prices > historical_prices * (1 + threshold)).sum()
sell_signals = (predicted_prices < historical_prices * (1 - threshold)).sum()
hold_signals = len(historical_prices) - buy_signals - sell_signals

# Total signals
total_signals = buy_signals + sell_signals + hold_signals
sell = (buy_signals / total_signals) * 100
buy = (sell_signals / total_signals) * 100
hold = (hold_signals / total_signals) * 100

# buy = 50
# sell = 30
# hold = 20

# Plot customization
fig = go.Figure()

# Add historical data trace
fig.add_trace(go.Scatter(
    x=historical_data['Date'],
    y=historical_data['Price'],
    mode='lines',
    name='Historical Prices',
    line=dict(color='cyan', width=2),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Add forecasted data trace
fig.add_trace(go.Scatter(
    x=forecast_data['Date'],
    y=forecast_data['Price'],
    mode='lines',
    name='Forecasted Prices',
    line=dict(color='magenta', width=2, dash='dash'),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Add a horizontal bar chart for Buy, Sell, Hold
fig.add_trace(go.Bar(
    x=[buy, sell, hold],  # Values
    y=['Buy', 'Sell', 'Hold'],  # Categories
    orientation='h',  # Horizontal bar chart
    marker=dict(color=['#90EE90', '#FFA07A', '#87CEEB']),
    opacity=0.7,
    xaxis='x2',
    yaxis='y2',
    showlegend=False
))

# fig.add_trace(go.Bar(
#     x=[buy, sell, hold],  # Values
#     y=['Buy', 'Sell', 'Hold'],  # Categories
#     orientation='h',  # Horizontal bar chart
#     marker=dict(
#         color=['#90EE90', '#FFA07A', '#87CEEB'],  # Subtle pastel colors
#         line=dict(
#             color=['#50C878', '#FF6347', '#4682B4'],  # Slightly darker outlines
#             width=2  # Outline thickness for the curvy effect
#         ),
#         opacity=0.85  # Slight transparency for subtlety
#     ),
#     width=0.4,  # Slimmer bars for aesthetics
#     showlegend=False
# ))

# Update layout for dark theme and bar chart positioning
fig.update_layout(
    title=dict(
        text='30-Day Stock Price Forecast',
        font=dict(size=24, color='white'),
        x=0.5
    ),
    xaxis=dict(
        title='Date',
        showgrid=False,
        gridcolor='gray',
        color='white',
        tickformat='%b %d',
        domain=[0, 0.7]  # Shrink main plot horizontally to free space
    ),
    yaxis=dict(
        title='Stock Price',
        showgrid=False,
        gridcolor='gray',
        color='white'
    ),
    xaxis2=dict(
        domain=[0.75, 1],  # Position bar chart on the right side
        anchor='y2',
        title='Proportion (%)',
        color='white',
        showline=False
    ),
    yaxis2=dict(
        domain=[0.3, 0.7],  # Center vertically relative to the main plot
        anchor='x2',
        color='white',
        showline=False
    ),
    paper_bgcolor='black',  # Dark background
    plot_bgcolor='black',
    legend=dict(
        font=dict(size=12, color='white'),
        bgcolor='rgba(0,0,0,0.5)',
        x=0.75,  # Position legend near the bar chart
        y=0.95
    )
)

# Show the plot
fig.show()


In [19]:
import plotly.graph_objects as go
import pandas as pd

# Historical Data
historical_data = pd.DataFrame({
    'Date': dates,  # Use fixed datetime dates
    'Price': df['Close']  # Historical prices
})
historical_data = historical_data.iloc[-7:]

# Forecast Data
forecast_data = pd.DataFrame({
    'Date': forecast_dates_7,  # Generated future dates
    'Price': forecasted_prices_original_7  # Forecasted prices
})

# Example proportions for Buy, Sell, Hold
historical_prices = historical_data['Price'].values
predicted_prices = forecast_data['Price'].values
threshold = 0.05  # 2% threshold

# Compute buy, sell, hold
buy_signals = (predicted_prices > historical_prices * (1 + threshold)).sum()
sell_signals = (predicted_prices < historical_prices * (1 - threshold)).sum()
hold_signals = len(historical_prices) - buy_signals - sell_signals

# Total signals
total_signals = buy_signals + sell_signals + hold_signals
sell = (buy_signals / total_signals) * 100
buy = (sell_signals / total_signals) * 100
hold = (hold_signals / total_signals) * 100

# buy = 50
# sell = 30
# hold = 20

# Plot customization
fig_1 = go.Figure()

# Add historical data trace
fig_1.add_trace(go.Scatter(
    x=historical_data['Date'],
    y=historical_data['Price'],
    mode='lines',
    name='Historical Prices',
    line=dict(color='cyan', width=2),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Add forecasted data trace
fig_1.add_trace(go.Scatter(
    x=forecast_data['Date'],
    y=forecast_data['Price'],
    mode='lines',
    name='Forecasted Prices',
    line=dict(color='magenta', width=2, dash='dash'),
    hoverlabel=dict(bgcolor="black", font_color="white")
))

# Add a horizontal bar chart for Buy, Sell, Hold
fig_1.add_trace(go.Bar(
    x=[buy, sell, hold],  # Values
    y=['Buy', 'Sell', 'Hold'],  # Categories
    orientation='h',  # Horizontal bar chart
    marker=dict(color=['#90EE90', '#FFA07A', '#87CEEB']),
    opacity=0.7,
    xaxis='x2',
    yaxis='y2',
    showlegend=False
))

# fig_1.add_trace(go.Bar(
#     x=[buy, sell, hold],  # Values
#     y=['Buy', 'Sell', 'Hold'],  # Categories
#     orientation='h',  # Horizontal bar chart
#     marker=dict(
#         color=['#90EE90', '#FFA07A', '#87CEEB'],  # Subtle pastel colors
#         line=dict(
#             color=['#50C878', '#FF6347', '#4682B4'],  # Slightly darker outlines
#             width=2  # Outline thickness for the curvy effect
#         ),
#         opacity=0.85  # Slight transparency for subtlety
#     ),
#     width=0.4,  # Slimmer bars for aesthetics
#     showlegend=False
# ))

# Update layout for dark theme and bar chart positioning
fig_1.update_layout(
    title=dict(
        text='7-Day Stock Price Forecast',
        font=dict(size=24, color='white'),
        x=0.5
    ),
    xaxis=dict(
        title='Date',
        showgrid=False,
        gridcolor='gray',
        color='white',
        tickformat='%b %d',
        domain=[0, 0.7]  # Shrink main plot horizontally to free space
    ),
    yaxis=dict(
        title='Stock Price',
        showgrid=False,
        gridcolor='gray',
        color='white'
    ),
    xaxis2=dict(
        domain=[0.75, 1],  # Position bar chart on the right side
        anchor='y2',
        title='Proportion (%)',
        color='white',
        showline=False
    ),
    yaxis2=dict(
        domain=[0.3, 0.7],  # Center vertically relative to the main plot
        anchor='x2',
        color='white',
        showline=False
    ),
    paper_bgcolor='black',  # Dark background
    plot_bgcolor='black',
    legend=dict(
        font=dict(size=12, color='white'),
        bgcolor='rgba(0,0,0,0.5)',
        x=0.75,  # Position legend near the bar chart
        y=0.95
    )
)

# Show the plot
fig_1.show()


In [20]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

# Assuming 'dates' is a pandas datetime series for the stock's historical data
# Assuming 'df' is a DataFrame with stock data (including 'Close' column)
historical_data = pd.DataFrame({
    'Date': dates,  # Last 30 days of dates
    'Price': df['Close']  # Last 30 days of stock prices
})
historical_data = historical_data.iloc[-60:]
# # Example forecasted data (replace with your actual forecast data)
# forecast_dates = pd.date_range(start="2024-12-15", periods=30, freq='D')  # Forecasted dates for the next 30 days # Replace with your actual forecasted prices

# Create forecast data DataFrame
forecast_data = pd.DataFrame({
    'Date': forecast_dates,  # Future dates
    'Price': forecasted_prices_original  # Forecasted prices
})

# Fetch Nifty 50 and S&P 500 data for the last 30 days
nifty_data = yf.Ticker("^NSEI")  # Nifty 50 index
sp500_data = yf.Ticker("^BSESN")  # S&P 500 index

nifty_hist = nifty_data.history(period="3mo")  # Fetch last 30 days of data
sp500_hist = sp500_data.history(period="3mo")  # Fetch last 30 days of data

# Align data to ensure all data frames have the same date range
nifty_hist = nifty_hist[['Close']].rename(columns={'Close': 'Nifty'})
sp500_hist = sp500_hist[['Close']].rename(columns={'Close': 'S&P 500'})

# Combine all the data into one DataFrame
combined_data = pd.concat([historical_data.set_index('Date'), nifty_hist, sp500_hist], axis=1)

# Display the combined data
print(combined_data.tail())

# Plotting
fig_2 = go.Figure()

# Add stock data trace
fig_2.add_trace(go.Scatter(x=historical_data['Date'], y=historical_data['Price'], mode='lines', name='Stock Price', line=dict(color='blue')))

# Add forecast data trace (future prediction, styled differently)
fig_2.add_trace(go.Scatter(x=forecast_data['Date'], y=forecast_data['Price'], mode='lines', name='Forecasted Stock', line=dict(color='orange', dash='dot')))

# # Add Nifty 50 data trace
fig_2.add_trace(go.Scatter(x=nifty_hist.index, y=nifty_hist['Nifty'], mode='lines', name='Nifty 50', line=dict(color='green', dash='dash')))

# Add S&P 500 data trace
# fig_2.add_trace(go.Scatter(x=sp500_hist.index, y=sp500_hist['S&P 500'], mode='lines', name='S&P 500', line=dict(color='red', dash='dash')))

# Customize the layout
fig_2.update_layout(
    title="Stock vs Nifty 50 (Last 30 Days)",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    showlegend=True
)

fig_2.show()


                           Price         Nifty       S&P 500
Date                                                        
2024-11-27 18:30:00+00:00    NaN  23914.150391  79043.742188
2024-11-28 18:30:00+00:00    NaN  24131.099609  79802.789062
2024-12-01 18:30:00+00:00    NaN  24276.050781  80248.078125
2024-12-02 18:30:00+00:00    NaN  24457.150391  80845.750000
2024-12-03 18:30:00+00:00    NaN  24467.449219  80956.328125


In [21]:
from transformers import pipeline

# Load a pre-trained language model (FLAN-T5 or similar)
# generator = pipeline("text-generation", model="google/flan-t5-large")

# Prompt for generating stock insights
prompt = """
Analyze the performance of the Reliance stock (RELIANCE.NS) over the last quarter. Include growth trends, peer comparison, and insights for shareholders.
"""

# # Generate insights
# output = generator(prompt, max_length=150, num_return_sequences=1)

# # Display the generated insights
# print(output[0]['generated_text'])


/Users/aayush/Foundation-Project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [22]:
from text_generation.inference_api import deployed_models

print(deployed_models())

[DeployedModel(model_id='meta-llama/Llama-3.2-11B-Vision-Instruct', sha='9eb2daaa8597bf192a8b0e73f848f3a102794df5'), DeployedModel(model_id='01-ai/Yi-1.5-34B-Chat', sha='fa4ffba162f20948bf77c2a30eca952bf0812b7f'), DeployedModel(model_id='bigcode/octocoder', sha='0f863c63e38ba80fc2c4010f34a7f46d537a9eee'), DeployedModel(model_id='bigcode/santacoder', sha='bb3be599767d93ce716293e9193c027e855a9524'), DeployedModel(model_id='bigcode/starcoder', sha='827cd7721231872c152e3faef5cae1b22964ed58'), DeployedModel(model_id='bigcode/starcoder2-15b', sha='46d44742909c03ac8cee08eb03fdebce02e193ec'), DeployedModel(model_id='bigcode/starcoder2-3b', sha='733247c55e3f73af49ce8e9c7949bf14af205928'), DeployedModel(model_id='bigcode/starcoderplus', sha='c2c2f1397ed0caf52353abe14e35120af37a6d6b'), DeployedModel(model_id='codellama/CodeLlama-13b-hf', sha='8da65ff4ee20f74ecd107ca9d54f9f121b279860'), DeployedModel(model_id='codellama/CodeLlama-34b-Instruct-hf', sha='d4c1c474abcacd32d2a6eda45f9811d38c83e93d'), D

In [23]:
import yfinance as yf

def get_stock_news_yfinance(ticker):
    # Fetch the stock data
    stock = yf.Ticker(ticker)
    
    # Fetch the news
    try:
        news = stock.news  # Retrieve news articles
    except AttributeError:
        return "Failed to retrieve news. Ensure the `yfinance` version supports this feature."

    if not news:
        return "No news articles found for this stock."

    # Format the news articles
    formatted_news = []
    for article in news[:5]:  # Limit to top 5 news articles
        title = article.get("title", "No title available")
        link = article.get("link", "No link available")
        publisher = article.get("publisher", "Unknown source")
        formatted_news.append(f"Title: {title}\nPublisher: {publisher}\nLink: {link}\n")
    
    return formatted_news

# Example: Fetch news for Reliance Industries
ticker = "RELIANCE.NS"  # Yahoo Finance ticker for Reliance Industries
news = get_stock_news_yfinance(ticker)

# Display the news
if isinstance(news, list):
    for idx, article in enumerate(news, 1):
        print(f"News {idx}:\n{article}\n")
else:
    print(news)


News 1:
Title: Temu told to suspend Vietnam operations, state media reports
Publisher: Reuters
Link: https://finance.yahoo.com/news/temu-told-suspend-vietnam-operations-022053659.html


News 2:
Title: Here's how a US trade war between Mexico, Canada, and China could play out for the stock market
Publisher: Business Insider
Link: https://finance.yahoo.com/m/5196071d-b6a6-3514-93a9-2f778bad24d1/here%27s-how-a-us-trade-war.html


News 3:
Title: Eagle Point Credit Company Inc. Prices $100 Million Public Offering of Notes
Publisher: Business Wire
Link: https://finance.yahoo.com/news/eagle-point-credit-company-inc-021500744.html


News 4:
Title: Epic Airboat Tours Launches New Everglades Gator Experiences with More Options and Brand-New Vessel
Publisher: Newsfile
Link: https://finance.yahoo.com/news/epic-airboat-tours-launches-everglades-021500867.html


News 5:
Title: Correction to Press Release Announcing nCino’s Third Quarter Fiscal Year 2025 Financial Results
Publisher: GlobeNewswire
Lin

In [24]:
import io
from botocore.exceptions import NoCredentialsError
from plotly.io import write_image
FILE_NAME = f"Results/{vertical}/{ticker}/30_day_forecast.png"  # The name for the file in the S3 bucket
FILE_NAME_1 = f"Results/{vertical}/{ticker}/7_day_forecast.png"  # The name for the file in the S3 bucket
FILE_NAME_2 = f"Results/{vertical}/{ticker}/nifty_compare.png"  # The name for the file in the S3 bucket

# Function to upload file to S3
def upload_to_s3(file_data, bucket, file_name):
    s3 = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_DEFAULT_REGION
    )
    try:
        s3.upload_fileobj(file_data, bucket, file_name)
        print(f"File uploaded successfully to s3://{bucket}/{vertical}/{ticker}/{file_name}")
    except NoCredentialsError:
        print("AWS credentials not available.")
    except Exception as e:
        print(f"Error uploading file: {e}")

# Save Plotly figure to S3
def save_plot_to_s3(fig, bucket, file_name):
    # Convert Plotly figure to an image buffer
    buffer = io.BytesIO()
    fig.write_image(buffer, format="png")  # You need Orca installed for this
    buffer.seek(0)  # Reset buffer position

    # Upload to S3
    upload_to_s3(buffer, bucket, file_name)
save_plot_to_s3(fig, bucket_name, FILE_NAME)
save_plot_to_s3(fig_1, bucket_name, FILE_NAME_1)
save_plot_to_s3(fig_2, bucket_name, FILE_NAME_2)

File uploaded successfully to s3://isbfinanceproject/Energy/RELIANCE.NS/Results/Energy/RELIANCE.NS/30_day_forecast.png
File uploaded successfully to s3://isbfinanceproject/Energy/RELIANCE.NS/Results/Energy/RELIANCE.NS/7_day_forecast.png
File uploaded successfully to s3://isbfinanceproject/Energy/RELIANCE.NS/Results/Energy/RELIANCE.NS/nifty_compare.png
